In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
import sys, os
sys.path.append("../module/")
from lipinski import * 

In [3]:
con_odn = sqlite3.connect("../data/ligantes_odn.db")
con_k777 = sqlite3.connect("../data/ligantes_k777.db")

In [37]:
odn = pd.read_sql("select * from novos_ligantes", con_odn)

In [38]:
k777 = pd.read_sql("select * from novos_ligantes", con_k777)

In [16]:
def lipinski_trial(dataframe):
  '''
  Returns which of Lipinski's rules a molecule has failed, or an empty list
  
  Lipinski's rules are:
  Hydrogen bond donors <= 5
  Hydrogen bond acceptors <= 10
  Molecular weight < 500 daltons
  logP < 5
  '''
  violacoes_list = []
  for index, rows in  dataframe.iterrows():
    viol = 0
    if dataframe.iloc[index].hbd > 5:
        viol += 1
    if dataframe.iloc[index].hba > 10:
        viol += 1
    if dataframe.iloc[index].molwt >= 500:
        viol += 1
    if dataframe.iloc[index].logp >= 5:
        viol += 1
    violacoes_list.append(viol)
  return violacoes_list

In [41]:
odn['violacoes_lipinski'] = lipinski_trial(odn)

In [42]:
k777['violacoes_lipinski'] = lipinski_trial(k777)

In [54]:
odn_50 = odn.loc[(odn.Smiles.str.contains('#')) & (odn.i_tanimoto > 0.4) & (odn.violacoes_lipinski <= 1)].sort_values(by='active_probability', ascending=False)[:50]

In [55]:
k777_50 = k777.loc[(k777.Smiles.str.contains('#')) & (k777.i_tanimoto > 0.4) & (k777.violacoes_lipinski <= 1)].sort_values(by='active_probability', ascending=False)[:50]

In [56]:
k777_50.Smiles.to_csv("smiles_semelhantes_k777.smi",index = False, header = False)

In [57]:
odn_50.Smiles.to_csv("smiles_semelhantes_odn.smi",index = False, header = False)